In [1]:
import pandas as pd
import numpy as np
import openpyxl
import re
from re import sub
from decimal import Decimal
import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from numpy import unique, where
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.mixture import GaussianMixture
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from sklearn.metrics import r2_score, silhouette_samples, silhouette_score , completeness_score , homogeneity_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from IPython.display import display, HTML, display_html
import pickle

In [2]:
random_state = 0

In [3]:
airbnb_ny_listing  = '../../Data/LA_Airbnb/listings_detailed.csv'

In [4]:
# Extreme prices of the listing might skew or render prediction ineffective
def drop_outliers_IQR(df):
    iqr = df['price'].quantile(0.75) - df['price'].quantile(0.25)
    lower_limit = df['price'].quantile(0.25) - 1.5*iqr
    upper_limit = df['price'].quantile(0.75) + 1.5*iqr
    final_df = df[(df['price']>=lower_limit) & (df['price']<=upper_limit)]
    
    return final_df

# Data Cleaning
df_listing = pd.read_csv(airbnb_ny_listing)
df_listing['price'] = df_listing['price'].apply(lambda x: float(Decimal(sub(r'[^\d.]', '', x))))
df_listing['neighbourhood_cleansed'] = df_listing['neighbourhood_cleansed'].apply(lambda x: str(x).lower())
df_listing['amenities'] = df_listing['amenities'].apply(lambda x: str(x).lower())
df_listing['property_type'] = df_listing['property_type'].apply(lambda x: str(x).lower())
df_listing['room_type'] = df_listing['room_type'].apply(lambda x: str(x).lower())
df_listing['bathrooms_text'] = df_listing['bathrooms_text'].fillna(0)
df_listing['bathrooms_text'] = df_listing['bathrooms_text'].apply(lambda x: str(x).lower())
df_listing['bedrooms'] = df_listing['bedrooms'].fillna(0)
df_listing['beds'] = df_listing['beds'].fillna(0)
df_listing['latitude'] = np.round(df_listing['latitude'],5)
df_listing['longitude'] = np.round(df_listing['longitude'],5)
# df_listing = drop_outliers_IQR(df_listing)
# Reset the index the dataframe to match the index of countvectorizer to df in the later part
df_listing = df_listing.reset_index().drop(columns = ['index'])

len(df_listing)

40438

In [5]:
property_features = ['id', 'latitude','longitude', 'neighbourhood_cleansed', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities','minimum_nights', 'maximum_nights', 'price'] 
# Need to read documentation for meaning
# , 'minimum_minimum_nights','maximum_minimum_nights', 'minimum_maximum_nights',
# 'maximum_maximum_nights', 'minimum_nights_avg_ntm','maximum_nights_avg_ntm']

In [6]:
df_listing[property_features].head(2)

,id,latitude,longitude,neighbourhood_cleansed,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,minimum_nights,maximum_nights,price
0,65467,34.02438,-118.38374,culver city,private room in home,private room,2,NaN,2 baths,1.0,1.0,"[""wifi"", ""tv with standard cable"", ""long term ...",60,730,300.0
1,206662,34.10420,-118.34748,hollywood hills west,private room in condo,private room,1,NaN,1 shared bath,1.0,2.0,"[""tv with standard cable"", ""hot water"", ""first...",30,30,46.0


In [7]:
# Source for list of amenities
# https://towardsdatascience.com/predicting-airbnb-prices-with-deep-learning-part-1-how-to-clean-up-airbnb-data-a5d58e299f6c
# Airbnb standard amenities list extracted from host account
# compund a list of amenities based on both sources above 
# Realised amenities made up of generic and customised items for usually brands after going through the amenities section
# of the data set, making it hard to build a generalisable features for each listings

#####################
# NOTE: To be convert into a text file for ease of extraction
#####################
raw_amenities_list = ["24-hour check-in",'Accessible-height bed','Accessible-height toilet','Air conditioning','Air purifier',
 'Alfresco bathtub','Amazon Echo','Apple TV','BBQ grill','Baby bath','Baby monitor','Babysitter recommendations',
 'Balcony','Bath towel','Bathroom essentials','Bathtub','Bathtub with bath chair','Beach essentials',
 'Beach view','Beachfront','Bed linens','Bedroom comforts','Bidet','Body soap','Breakfast','Breakfast bar','Breakfast table',
 'Building staff','Buzzer/wireless intercom','Cable TV','Carbon monoxide detector','Cat','Ceiling fan','Ceiling hoist',
 'Central air conditioning','Changing table',"Chef's kitchen",'Children’s books and toys','Children’s dinnerware',
 'Cleaning before checkout','Coffee maker','Convection oven','Cooking basics','Crib','DVD player','Day bed',
 'Dining area','Disabled parking spot','Dishes and silverware','Dishwasher','Dog','Doorman','Double oven','Dryer',
 'EV charger','Electric profiling bed','Elevator','En suite bathroom','Espresso machine','Essentials','Ethernet connection',
 'Exercise equipment','Extra pillows and blankets','Family/kid friendly','Fax machine','Fire extinguisher','Fire pit',
 'Fireplace guards','Firm mattress','First aid kit','Fixed grab bars for shower','Fixed grab bars for toilet',
 'Flat path to front door','Formal dining area','Free parking on premises','Free street parking','Full kitchen','Game console',
 'Garden or backyard','Gas oven','Ground floor access','Gym','HBO GO','Hair dryer','Hammock','Handheld shower head',
 'Hangers','Heat lamps','Heated floors','Heated towel rack','Heating','High chair','High-resolution computer monitor',
 'Host greets you','Hot tub','Hot water','Hot water kettle','Indoor fireplace','Internet','Iron','Ironing Board',
 'Jetted tub','Keypad','Kitchen','Kitchenette','Lake access','Laptop friendly workspace','Lock on bedroom door',
 'Lockbox','Long term stays allowed','Luggage dropoff allowed','Memory foam mattress','Microwave','Mini fridge',
 'Mobile hoist','Mountain view','Mudroom','Murphy bed','Netflix','Office','Other','Other pet(s)','Outdoor kitchen',
 'Outdoor parking','Outdoor seating','Outlet covers','Oven','Pack ’n Play/travel crib','Paid parking off premises',
 'Paid parking on premises','Patio or balcony','Pets allowed','Pets live on this property','Pillow-top mattress',
 'Pocket wifi','Pool','Pool cover','Pool with pool hoist','Printer','Private bathroom','Private entrance',
 'Private gym','Private hot tub','Private living room','Private pool','Projector and screen','Propane barbeque','Rain shower',
 'Refrigerator','Roll-in shower','Room-darkening shades','Safe','Safety card','Sauna','Security system','Self check-in',
 'Shampoo','Shared gym','Shared hot tub','Shared pool','Shower chair','Single level home','Ski-in/Ski-out','Smart TV',
 'Smart lock','Smoke detector','Smoking allowed','Soaking tub','Sound system','Stair gates','Stand alone steam shower',
 'Standing valet','Steam oven','Step-free access','Stove','Suitable for events','Sun loungers','TV','Table corner guards',
 'Tennis court','Terrace','Toilet paper','Touchless faucets','Walk-in shower','Warming drawer','Washer','Washer / Dryer',
 'Waterfront','Well-lit path to entrance','Wheelchair accessible','Wide clearance to bed','Wide clearance to shower',
 'Wide doorway','Wide entryway','Wide hallway clearance','Wifi','Window guards','Wine cooler','toilet','Bath',
 'Bidet','Body soap','Cleaning products','Conditioner','Hair dryer','Hot water','Outdoor shower','Shampoo','Shower gel',
 'Essentials','Bed linen','Clothes storage','Dryer','Clothes drying rack','Extra pillows and blankets','Hangers','Iron',
 'Mosquito net','Room-darkening shades','Safe','Washing machine','Arcade games','Batting cage','Books and reading material',
 'Bowling alley','Climbing wall','Ethernet connection','Exercise equipment','Games console','Laser tag','Life-size games',
 'Mini golf','Cinema','Piano','Ping pong table','Pool table','Record player','Skate ramp','Sound system','Theme room',
 'TV','Baby bath','Baby monitor','Children’s bikes','Children’s playroom','Baby safety gates','Babysitter recommendations',
 'Board games','Changing table','Children’s books and toys','Children’s tableware','Cot','Fireplace guards','High chair',
 'Outdoor playground','Plug socket covers','Travel cot','Table corner guards','Window guards', 'Air conditioning',
 'Ceiling fan','Heating','Indoor fireplace','Portable fans','Carbon monoxide alarm','Fire extinguisher', 'First aid kit',
 'Smoke alarm','Dedicated workspace','Pocket wifi','Wifi','Baking sheet','Barbecue utensils','Bread maker','Blender',
 'Coffee','Coffee maker','Cooking basics','Dining table','Dishes and silverware','Dishwasher','Freezer','Kettle','Kitchen',
 'Kitchenette','Microwave','Mini fridge','Oven','Refrigerator','Rice cooker','Stove','Toaster','Waste compactor','Wine glasses',
 'Beach access','Lake access','Launderette nearby','Private entrance','Resort access','Ski-in/Ski-out','Waterfront',
 'Garden','BBQ grill','Beach essentials','Bikes','Boat berth','Fire pit','Hammock','Kayak','Outdoor dining area',
 'Outdoor furniture','Outdoor kitchen','Patio or balcony','Sun loungers','Lift','EV charger','Free parking on premises',
 'Hockey rink','Free on-street parking','Gym','Hot tub','Paid parking off premises','Paid parking on premises','Pool','Sauna',
 'Single level home','Breakfast','Cleaning available during stay','Long-term stays allowed','Luggage drop-off allowed']

len(raw_amenities_list)

330

In [8]:
# Get the unique amenities available in Airbnb
raw_amenities_list = [i.lower() for i in raw_amenities_list]
amenities_universe = np.unique(raw_amenities_list)

In [9]:
# Use regex to convert the string of amenities into individual string object to check if the object is in the amenities universe
pattern = r'\"(.*?)\"'
# This step ignores the idea that there might be more than 1 amenities
df_listing['amenities_clean'] = df_listing['amenities'].apply(lambda x: [i for i in re.findall(pattern,x) if i in amenities_universe])
# Convert list of amenities for each property for subsequent use of Vectoriser
df_listing['amenities_clean_vec'] = df_listing['amenities_clean'].apply(lambda x: '"'.join(x))
# Count the number of amenities listed for the property
df_listing['amenities_count'] = df_listing['amenities_clean'].apply(lambda x: len(x))

In [10]:
# Count vectorizer - This step will naturally collate the full list of amenities based on the detail dataset
#                    which is a subset of the amenities_universe
# Use regex to tokenize the string for count vectorizer
pattern = r'\"(.*?)\"'
vectorizer_count = CountVectorizer(token_pattern = pattern)
property_amenities_list = [i for i in df_listing['amenities_clean_vec']]

property_features_count_vectorized = vectorizer_count.fit_transform(property_amenities_list)
amenities_feature_count_name_clean = vectorizer_count.get_feature_names_out()
amenities_feature_count_data_clean = property_features_count_vectorized.toarray()
amenities_feature_count_df = pd.DataFrame(amenities_feature_count_data_clean,columns = amenities_feature_count_name_clean)
amenities_feature_count_df.tail()

,air conditioning,apple tv,baby bath,baby monitor,baby safety gates,babysitter recommendations,baking sheet,balcony,barbecue utensils,bathroom essentials,...,tennis court,terrace,toaster,tv,washer,waterfront,wifi,window guards,wine cooler,wine glasses
40433,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
40434,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
40435,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
40436,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,1
40437,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# Bathroom
# 3 new columns:
#   - bathroom_count
#   - bathroom_type: Assumed to be "private" if "shared" is not mentioned 
#   - bathrooms_type_code: shared=0 & private=1
float_pattern = r"[-+]?(?:\d*\.*\d+)"

df_listing['bathrooms_count'] = df_listing['bathrooms_text'].apply(lambda x: re.findall(float_pattern,str(x)) if x != 'half' else [0.5])
df_listing['bathrooms_count'] = df_listing['bathrooms_count'].apply(lambda x: x[0] if len(x)>0 else 0)
df_listing['bathrooms_type'] = df_listing['bathrooms_text'].apply(lambda x: 'shared' if 'shared' in str(x) else 'private')
df_listing['bathrooms_type_code'] = np.where(df_listing['bathrooms_type']=='shared',0,1)

In [12]:
# Property and room type
# 3 new columns:
#   - property_type_clean: Remove qualitative words in the "prohibitedwords" list to identify unique property type
#   - property_type_code: Assign a number to each property type
#   - room_type_code: Assign a number to each room type (This offers qualitative description for the property type i.e. "private","shared")
prohibitedWords = ['private', 'shared', ' in ', 'entire', 'room'] # added spaces in front and behind 'in' to ensure accuracy
big_regex = re.compile('|'.join(map(re.escape, prohibitedWords)))
df_listing['property_type_clean'] =  df_listing['property_type'].apply(lambda x: big_regex.sub("", x).strip())
df_listing['property_type_clean'] = np.where(df_listing['property_type_clean']=='',df_listing['room_type'],df_listing['property_type_clean'])

# create dictionary for property type code
# property_type_list = list(np.unique(df_listing['property_type_clean']))
# property_type_dict = dict(zip(property_type_list,np.arange(len(property_type_list))))

# # create dictionary for room type code
# room_type_list = list(np.unique(df_listing['room_type']))
# room_type_dict = dict(zip(room_type_list,np.arange(len(room_type_list))))

# # Assign property type code
# df_listing['property_type_code'] = df_listing['property_type_clean'].apply(lambda x: property_type_dict[x])

# # Assign room type code
# df_listing['room_type_code'] = df_listing['room_type'].apply(lambda x: room_type_dict[x])



In [13]:
# neighbourhood_cleansed
# 1 new column:
#    - neighbourhood_cleansed_code: index the neighbourhood_cleansed column to further segregate the property according to the location

# neighbourhood_cleansed_dict = dict(zip(np.unique(df_listing['neighbourhood_cleansed']),np.arange(len(np.unique(df_listing['neighbourhood_cleansed'])))))
# df_listing['neighbourhood_cleansed_code'] = df_listing['neighbourhood_cleansed'].apply(lambda x: neighbourhood_cleansed_dict[x])





In [14]:
# Recontruct the final dataframe for analysis
required_column_list = ['price','latitude','longitude','property_type_clean','room_type', 'neighbourhood_cleansed',
                        'bedrooms', 'beds','bathrooms_type','bathrooms_count','minimum_nights', 'maximum_nights',
                        'amenities_count',]

final_df = df_listing[required_column_list].join(amenities_feature_count_df)
# Drop the outliers
# final_df = drop_outliers_IQR(final_df)

print("Number of features {}".format(len(final_df.columns)-1))
display(final_df.head())
# final_df.dtypes

Number of features 169


,price,latitude,longitude,property_type_clean,room_type,neighbourhood_cleansed,bedrooms,beds,bathrooms_type,bathrooms_count,...,tennis court,terrace,toaster,tv,washer,waterfront,wifi,window guards,wine cooler,wine glasses
0,300.0,34.02438,-118.38374,home,private room,culver city,1.0,1.0,private,2,...,0,0,0,0,0,0,0,0,0,0
1,46.0,34.10420,-118.34748,condo,private room,hollywood hills west,1.0,2.0,shared,1,...,0,0,0,0,0,0,1,0,0,0
2,140.0,34.00985,-118.40798,guesthouse,entire home/apt,culver city,1.0,1.0,private,1,...,0,0,0,1,0,0,0,0,0,0
3,340.0,34.05303,-118.39449,townhouse,entire home/apt,pico-robertson,3.0,5.0,private,3,...,0,0,0,0,1,0,0,0,0,0
4,115.0,33.98301,-118.38607,condo,entire home/apt,culver city,2.0,3.0,private,2,...,0,0,0,0,0,0,1,0,0,0


In [15]:
final_df.to_csv('amenities_features.csv',index=False)

In [54]:
list(final_df.columns)

['price',
 'latitude',
 'longitude',
 'property_type_clean',
 'room_type',
 'neighbourhood_cleansed',
 'bedrooms',
 'beds',
 'bathrooms_type_code',
 'bathrooms_count',
 'minimum_nights',
 'maximum_nights',
 'amenities_count',
 'air conditioning',
 'apple tv',
 'baby bath',
 'baby monitor',
 'baby safety gates',
 'babysitter recommendations',
 'baking sheet',
 'balcony',
 'barbecue utensils',
 'bathroom essentials',
 'bathtub',
 'bbq grill',
 'beach access',
 'beach essentials',
 'beach view',
 'beachfront',
 'bed linens',
 'bedroom comforts',
 'bidet',
 'bikes',
 'blender',
 'board games',
 'body soap',
 'books and reading material',
 'bread maker',
 'breakfast',
 'breakfast bar',
 'building staff',
 'cable tv',
 'carbon monoxide alarm',
 'ceiling fan',
 'central air conditioning',
 'changing table',
 "chef's kitchen",
 'cleaning available during stay',
 'cleaning before checkout',
 'cleaning products',
 'coffee',
 'coffee maker',
 'conditioner',
 'cooking basics',
 'crib',
 'dedicated